In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import plotly.express as px
import pycountry_convert as pc
import pylab as pl
import matplotlib.pyplot as plt

from countrygroups import G20
from countrygroups import NON_ANNEX_ONE
from countrygroups import ANNEX_ONE

In [2]:
link = r'/Users/jiaminlim/Documents/Research/climate_nlp/IPCCandPolicy/metadata.csv'

metadata = pd.read_csv(link)

# Using huggingface (easiest)
dataset = load_dataset("ClimatePolicyRadar/global-stocktake-documents")

# Using pandas
link = r'/Users/jiaminlim/Documents/Research/climate_nlp/global-stocktake-documents/full_text.parquet'
df = pd.read_parquet(link)

***

## Features

In [3]:
# Select relevant columns that you want
df = df[['geography_iso','author','document_name','date','type','type_confidence','text','text_block_id','types','document_source_url','translated']]

In [4]:
# Text field 
df['text'] = df['text'].str.replace('\n',' ')
df['text']  = df['text'].astype('str')

In [5]:
# Document name field
df["document_name"] =  df["document_name"].str.replace('\r','')

In [6]:
# Document types - dealing with multiple document types
df['types0'] = df.types.apply(lambda x: x[0])
df['n_types'] = df.types.apply(lambda x: len(x)) # maximum two 'types'
df['types1'] = df.types.apply(lambda x: x[1] if len(x)>1 else 'n/a')

In [7]:
# Document types - adjusting spelling error
df['types0'] = df['types0'].replace('Global Stocktake Synthesis Reports', 'Global Stocktake Synthesis Report', regex=True)
df['types0'] = df['types0'].replace('National Communcation', 'National Communication', regex=True)

In [8]:
# Document types - creating broader categories
df['types0_agg'] = np.where((df['types0']=='Biennial Update Report')|(df['types0']=='Biennial Report'),
                            'Biennial Update Report/Biennial Report',
                            df['types0'])
df['types0_agg'] = np.where((df['types0_agg']=='National Communication')|(df['types0_agg']=='Nationally Determined Contribution')|(df['types0_agg']=='Long-Term Low-Emission Development Strategy')|(df['types0_agg']=='Long-Term Low-Emission Development Strategy')|(df['types0_agg']=='Biennial Update Report/Biennial Report')|(df['types0_agg']=='National Inventory Report'),
                            df['types0_agg'],
                            'Other')

In [65]:
# Author name - extracting first author
df['n_author'] = df.author.apply(lambda x: len(x))
df['first_author'] = df.author.apply(lambda x: x[0])

In [73]:
pd.DataFrame(df[['n_author','document_name']].groupby(['n_author']).nunique())

,document_name
n_author,
1,1476
2,39
3,7
4,1
5,3
6,2
7,1


In [74]:
df['document_name'].nunique()

1529

In [10]:
# Geography 
# Removing some incorrect entries from XAB tag in geography_iso, and tagging them with their relevant country code
df['geography_iso'] = np.where(df['first_author'] == 'India', 'IND', df['geography_iso'])
df['geography_iso'] = np.where(df['first_author'] == 'Bhutan', 'BTN', df['geography_iso'])

In [11]:
df['is_author_country'] = np.where(df['geography_iso'] != 'nan', True, False)

In [12]:
# Author - creating author type field
df['author_type'] = np.where(df['is_author_country'] == True, 'Country', 
                             np.where(df['first_author'] == 'IPCC', 'IPCC', 
                                 np.where(df['first_author'].str.contains('UNFCCC'), 'UNFCCC',
                                      'Other')))

In [13]:
# Date - add year and m-y column
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['month_year'] = df['date'].dt.to_period('M')
df["date_document_name"] = df["date"].astype(str) + " - " + df["document_name"]

In [14]:
df['n_words'] =  df['text'].str.count(' ') + 1
df['bool_ipcc'] =  df['text'].str.contains("IPCC")

In [15]:
df['text_startsLC'] = df['text'].astype(str).str[0].str.islower()

In [16]:
# Add country names and continents
def country_to_continent(country_alpha3):
    try:
        # country_name = pc.country_alpha3_to_country_name(country_alpha3)
        country_alpha2 = pc.country_alpha3_to_country_alpha2(country_alpha3)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    except:
            if country_alpha3 =='TLS':
                country_continent_name = "Asia"
            elif country_alpha3 =='UAE':
                country_continent_name = "Asia"
            elif country_alpha3 =='VAT':
                country_continent_name = "Europe"
            elif country_alpha3 =='EUR':
                country_continent_name = "Europe"
            else:
                country_continent_name = "null"
    return country_continent_name

In [17]:
def countrycode_to_name(country_alpha3):
    try:
        country_alpha2 = pc.country_alpha3_to_country_alpha2(country_alpha3)
        #print(country_alpha2)
        country_name = pc.country_alpha2_to_country_name(country_alpha2)
    except:
        country_name = "null"
    return country_name

In [18]:
def countryname_to_code3(cn_name):
    try:
        country_alpha3 = pc.country_name_to_country_alpha3(cn_name)
    except:
        country_alpha3 = "null"   
    return country_alpha3

In [19]:
df['Continent'] = df['geography_iso'].map(country_to_continent)

In [20]:
df['geography_iso_name'] = df['geography_iso'].map(countrycode_to_name)

In [21]:
df['UNFCCC_annex'] = np.where(df['author_type'] == 'Country',
                             np.where(df['geography_iso'].isin(ANNEX_ONE), 'Annex I', 
                                 np.where(df['geography_iso'].isin(NON_ANNEX_ONE), 'Annex II', 'Other')),'null')

In [22]:
df['G20'] = np.where(df['author_type'] == 'Country',
                             np.where(df['geography_iso'].isin(G20), True,False),'null')

In [55]:
df[['geography_iso','G20']].groupby(['geography_iso','G20']).size().head(50)

geography_iso  G20  
AFG            False     8214
AGO            False     5254
ALB            False     7571
AND            False     3208
ARE            False     1561
ARG            True     28568
ARM            False     5028
ATG            False     3365
AUS            True     12166
AUT            False     4511
AZE            False     4368
BDI            False     4039
BEL            False     5284
BEN            False     3784
BFA            False     3695
BGD            False     3301
BGR            False     4715
BHR            False     1252
BHS            False     1055
BIH            False     4225
BLR            False      614
BLZ            False     5216
BOL            False     4160
BRA            True      5508
BRB            False     1345
BRN            False       97
BTN            False     1761
BWA            False     1445
CAF            False     1457
CAN            True     13181
CHE            False     8766
CHL            False    19219
CHN            True

***

# Data sub-sets

In [23]:
filter_country = (df['author_type']=='Country')
filter_ipccreport = (df['author_type']=='IPCC')
filter_ipccmentions = (df['bool_ipcc']==True)

In [24]:
# Filters: References to IPCC reports
ref0 = (df['text'].str.startswith("IPCC."))
ref1 = (df['text'].str.startswith("IPCC ("))
ref2 = (df['text'].str.startswith("IPCC, "))
ref3 = (df['text'].str.startswith("Intergovernmental Panel on Climate Change (IPCC). "))
ref4 = (df['text'].str.contains("Contribution of Working Group")
# [Add justification in code] 
ref5 = (df['text'].str.contains("IPCC Special Report on the impacts of global warming of 1.5°C above pre"))

refs = (ref0 | ref1 | ref2 | ref3 | ref4 | ref5)
not_refs = (~ref0 & ~ref1 & ~ref2 & ~ref3 & ~ref4 & ~ref5)

In [26]:
# Filters: Body text
body0 = (~df['text'].str.startswith("Source:"))
body1 = (~df['text'].str.startswith("Legend: "))
body2 = (~df['text'].str.startswith("*"))
body3 = (df['type']=='Text')
body4 = (df['n_words']>10)
bodys = body0 & body1 & body1 & body2 & body3 & body4

## Dataset subset: Country Policy Reports

In [28]:
df_countrysubmissions = df[filter_country]
#df_countrysubmissions.to_csv('df_countrysubmissions.csv')
len(df_countrysubmissions)

1102177

## Dataset subset: Country Policy Reports IPCC mentions

In [30]:
df_ipccmentions = df[(filter_ipccmentions)& filter_country]
len(df_ipccmentions)

25948

In [31]:
df_ipccmentions['document_name'].nunique()

794

### Body

In [33]:
df_ipccmentions_body = df[(filter_ipccmentions)
                    & filter_country
                    & not_refs
                    & bodys]
len(df_ipccmentions_body)

18848

In [34]:
df_ipccmentions_body['document_name'].nunique()

750

In [35]:
df_ipccmentions_body.to_csv("df_ipccmentions_body.csv")
df_ipccmentions_body_readin = pd.read_csv("df_ipccmentions_body.csv")
len(df_ipccmentions_body_readin)

18848

In [56]:
df_ipccmentions_body['document_name'].value_counts()

document_name
Afghanistan. Biennial update report (BUR). BUR 1. National inventory report.     489
Argentina. Biennial update report (BUR). BUR 4.                                  467
Mexico. 2018 National Inventory Report (NIR).                                    430
Argentina. Biennial update report (BUR). BUR 4. National inventory report.       420
Argentina. Biennial update report (BUR). BUR 3. National inventory report.       406
                                                                                ... 
Updated Second Nationally Determined Contribution of the United Arab Emirates      1
Argentina Second NDC (Updated submission)                                          1
Italy's Adaptation communication                                                   1
Saint Kitts and Nevis First NDC (Updated submission)                               1
Eswatini First NDC (Archived)                                                      1
Name: count, Length: 750, dtype: int64

### References to IPCC report

In [37]:
df_ipccmentions_ipccrefs = df[(filter_ipccmentions)
                    & filter_country
                    & refs]
len(df_ipccmentions_ipccrefs)

817

In [38]:
df_ipccmentions_ipccrefs.to_csv('df_ipccmentions_ipccrefs.csv')
df_ipccmentions_ipccrefs_readin = pd.read_csv("df_ipccmentions_ipccrefs.csv")
len(df_ipccmentions_ipccrefs_readin)

817

In [39]:
df_ipccmentions_ipccrefs['document_name'].nunique()

243

## Dataset subset: IPCC report

In [40]:
df_ipccreport = df[(filter_ipccreport)]
len(df_ipccreport)

37745

In [41]:
df_ipccreport['document_name'].nunique()

52

### Body

In [43]:
# These phrases are used to pick up the text strings that contain a list of authors
r0 = df_ipccreport['text'].str.contains("Association of Polar Early Career Scientist \(APECS\)")
r1 = df_ipccreport['text'].str.contains("YESS \(Young Earth System Scientists community\)")
r2 = df_ipccreport['text'].str.contains("United Kingdom \(of Great Britain and Northern Ireland\)")
r3 = df_ipccreport['text'].str.contains("Permafrost Young Research Network \(PYRN\)")
r4 = df_ipccreport['text'].str.contains("\(USA\)")
r5 = df_ipccreport['text'].str.contains("\(United States of America\)")
r6 = df_ipccreport['text'].str.contains("\(The United States of America\)")
r7 = df_ipccreport['text'].str.contains("\(the United States of America\)")
r8 = df_ipccreport['text'].str.contains("\(United Kingdom\)")
r9 = df_ipccreport['text'].str.contains("\(France\)")
r10 = df_ipccreport['text'].str.contains("\(Germany\)")
remove_authors = r0 | r1 | r2 | r3 | r4 | r5 | r6 | r7 | r8 | r9 | r10

In [44]:
df_ipccreport_body = df[(filter_ipccreport)
                    & not_refs
                    & bodys
                    & ~remove_authors]
len(df_ipccreport_body)

16666

In [51]:
df_ipccreport_body.to_csv('df_ipccreport_body.csv')
df_ipccreport_body_readin = pd.read_csv("df_ipccreport_body.csv")
len(df_ipccreport_body_readin)

16666

In [64]:
pd.DataFrame(df_ipccreport[['date','document_name']].groupby(['date']).size()).head(100)
# All the reports published by the IPCC since 2018 were included in the dataset, other than 2019 Refinement to the 2006 IPCC Guidelines for National Greenhouse Gas Inventories.
# This includes 3 special reports, the 4 AR6 related reports published between 2021 and 2023. 

,0
date,
2018-10-01,4259
2019-08-01,10481
2019-09-01,8123
2021-08-01,5552
2022-02-01,2714
2022-04-04,5075
2023-03-01,1541


### References to IPCC report

In [ ]:
df_ipccreport_ipccrefs = df[(filter_ipccreport)
                    & refs]
len(df_ipccreport_ipccrefs)

In [57]:
df_ipccreport_ipccrefs['document_name'].nunique()

NameError: name 'df_ipccreport_ipccrefs' is not defined

***

# Data Checks

### Word count

In [ ]:
df['n_words'].hist(bins=100)
max(df['n_words'])

In [ ]:
df_long = pd.DataFrame(df[df['n_words']>1000])
# pd.set_option('display.max_colwidth', None)
df_long

In [ ]:
# Words per document
wordcount = pd.DataFrame(df.groupby('date_document_name')['n_words'].sum())
wordcount['n_words'].hist(bins=100)

### Text count

In [ ]:
textcount = pd.DataFrame(df.groupby('geography_iso')['text'].nunique())
textcount = textcount.rename(columns={'text': 'n_text'})
textcount = textcount.reset_index()
i = textcount[((textcount.geography_iso == 'nan') &( textcount.n_text == 110271))].index
textcount = textcount.drop(i)
textcount.sort_values(['n_text'],ascending=False).head(20)

In [ ]:
fig = px.choropleth(textcount, locations="geography_iso",
                    color="n_text", 
                    #hover_name="geography_iso", 
                    title="Number of Text Rows by Country",
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

In [ ]:
textcount_time = df.groupby('year')['text'].nunique()
textcount_time.plot.bar()

### Document count

In [ ]:
# Unique documents per geography and report type
print(df['document_name'].nunique())
pivot_doccount= pd.pivot_table(data=df,index='geography_iso',columns='types0',values='document_name',aggfunc=lambda x: len(x.unique()),margins=True)
pivot_doccount

In [ ]:
doccount = pd.DataFrame(df.groupby('geography_iso')['document_name'].nunique())
doccount = doccount.rename(columns={'document_name': 'n_docs'})
doccount = doccount.reset_index()
i = doccount[((doccount.geography_iso == 'nan') &( doccount.n_docs == 512))].index
doccount = doccount.drop(i)
#docname.sort_values(['n_docs'],ascending=False).head(20)

In [ ]:
fig = px.choropleth(doccount, locations="geography_iso",
                    color="n_docs", 
                    #hover_name="geography_iso", 
                    title="Number of Documents by Country",
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

In [ ]:
doccount_type = pd.DataFrame(df.groupby('types0')['document_name'].nunique())
doccount_type.sort_values('document_name').plot.barh(legend=None)

In [ ]:
doccount_time = df.groupby('year')['document_name'].nunique()
doccount_time.plot.bar()

### Parsing

In [ ]:
# row per document
df_rowperdoc = df.groupby('document_name')['text'].count()
df_rowperdoc.describe()
#df_rowperdoc.sort_values(ascending=False).head(5)


In [ ]:
# words per country
df_wordspercountry = df[df['is_author_country'] == True].groupby('geography_iso')['text'].count()
df_rwordspercountry.describe()

In [ ]:
# documents per country
df_docspercountry = df[df['is_author_country'] == True].groupby('geography_iso')['document_name'].nunique()
df_docspercountry.describe()

In [ ]:
# histogram of words per row - count document
by_document = df[df['is_author_country'] == True].groupby(['document_name']).agg(n_words = ('n_words','sum'), n_rows_unique=('text','nunique'),n_rows_count=('text','count'))
by_document = by_document.reset_index()

by_document['Words/Row'] = by_document['n_words']/by_document['n_rows_count']
by_document['Ratio'] = by_document['n_rows_unique']/by_document['n_rows_count']

In [ ]:
output = by_document.sort_values('Words/Row',ascending = True).head(5)
output = output[['document_name','n_words','n_rows_count','n_rows_unique','Words/Row']]
output.to_latex()

In [ ]:
by_document.hist(column='Words/Row',bins = 50)
pl.title("Country Policy Documents - Document count by Words/Row")
pl.xlabel("Words/Row")
pl.ylabel("No. of Documents")

In [ ]:
by_document.describe()

In [ ]:
#by_document[by_document['document_name'].str.contains("Hungary")]
by_document.hist(column='Ratio',bins = 50)
pl.title("Histogram - Country Policy Reports by Words/Row")
pl.xlabel("Words/Row")
pl.ylabel("No. of Documents")

In [ ]:
plt.scatter(by_document['Ratio'], by_document['Words/Row'])
plt.title("Country Policy Reports")
plt.xlabel("Ratio of No. of Rows (Unique Text) to No. of Rows")
plt.ylabel("Words/Row")

plt.show()

In [ ]:
# Group by author_type and document type // count rows and documents
by_authoranddoc = df.groupby(['author_group']).agg(n_authors=('first_author','nunique'),n_rows=('text','count'),n_docs=('document_name','nunique'))
by_authoranddoc.loc['Total'] = by_authoranddoc.iloc[:, :].sum()

# by_authoranddoc.to_latex()
by_authoranddoc 

In [ ]:
# words per row
df_countrysubmissions['n_words'].describe()

In [ ]:
# row per document
df_rowperdoc = df_countrysubmissions.groupby('document_name')['text'].count()
df_rowperdoc.describe()

In [ ]:
# documents per country
df_docspercountry = df_countrysubmissions.groupby('first_author')['document_name'].nunique()
df_docspercountry.describe()